#### Loading Modules

In [1]:
'''This script loads pre-trained word embeddings (GloVe embeddings)
into a Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classication of newsgroup messages into 20 different categories).
GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)
20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html
Currently used dataset is a subset of it.
'''

from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Lambda
from keras.layers import Embedding
from keras.models import Sequential
import tensorflow as tf
import pandas as pd

BASE_DIR = os.getcwd()
GLOVE_DIR = BASE_DIR
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


Using TensorFlow backend.


#### (Optional) Loading Glove Vectors to create our own embedding matrix

In [3]:
print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found {} word vectors.'.format(len(embeddings_index)))


Indexing word vectors.


#### Loading Dataset

In [5]:
print('Processing text dataset')

raw_data = pd.read_csv('20news_raw.csv', encoding='latin-1')
labels_index = {}
for idx, i in enumerate(raw_data.categories.unique()):
    labels_index[i] = idx

print('Found %s texts.' % len(raw_data.texts))


Processing text dataset
Found 11314 texts.


#### Converting data into required format for a neural network

In [6]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(raw_data.texts)
sequences = tokenizer.texts_to_sequences(raw_data.texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = raw_data.categories
labels = [labels_index[label] for label in labels]

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 153445 unique tokens.
Shape of data tensor: (11314, 1000)
Shape of label tensor: (11314, 20)


#### Creating a test train split on a numpy array

In [7]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]


#### Creating Embedding Matrix as input for Neural Network

In [8]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


Preparing embedding matrix.


#### Creating Model architecture and training Model

In [10]:
print('Training model.')

model = Sequential()
model.add(Embedding(num_words,
                    EMBEDDING_DIM,
                    input_length=MAX_SEQUENCE_LENGTH,
                    weights=[embedding_matrix],
#                     trainable=False
                    ))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(len(labels_index), activation='softmax'))

from keras import callbacks
from keras.optimizers import RMSprop


rmsprop = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=64,
          epochs=20,
          validation_data=(x_val, y_val))


Training model.
Train on 9052 samples, validate on 2262 samples
Epoch 1/20
9052/9052 [==============================] - 5s - loss: 2.6618 - acc: 0.1891 - val_loss: 2.3752 - val_acc: 0.2480
Epoch 2/20
9052/9052 [==============================] - 5s - loss: 2.1584 - acc: 0.3290 - val_loss: 2.0492 - val_acc: 0.3187
Epoch 3/20
9052/9052 [==============================] - 5s - loss: 1.9036 - acc: 0.4021 - val_loss: 1.8228 - val_acc: 0.4151
Epoch 4/20
9052/9052 [==============================] - 5s - loss: 1.7485 - acc: 0.4401 - val_loss: 1.7175 - val_acc: 0.4372
Epoch 5/20
9052/9052 [==============================] - 5s - loss: 1.6339 - acc: 0.4771 - val_loss: 1.5813 - val_acc: 0.5102
Epoch 6/20
9052/9052 [==============================] - 5s - loss: 1.5492 - acc: 0.5046 - val_loss: 1.5357 - val_acc: 0.5062
Epoch 7/20
9052/9052 [==============================] - 4s - loss: 1.4770 - acc: 0.5318 - val_loss: 1.5449 - val_acc: 0.5035
Epoch 8/20
9052/9052 [==============================] - 4s - 

#### To get classification accuracy

In [12]:
model.evaluate(x_val, y_val)

2240/2262 [============================>.] - ETA: 0s

[1.187747670316148, 0.60919540235155145]

The first number in above list is the loss of batch and the second number is the accuracy